In [1]:
import pandas as pd
df  = pd.read_csv("./Financial-Literacy-project-master/dataset_with_score.csv", index_col=0)

In [2]:
df.head(1)

,P2,P3,P4,P5,P6,P7,P8,P9,P10,F1,...,K3,K4,K5,K6,K7,K8,K9,K10,K11,TOTAL
id,,,,,,,,,,,,,,,,,,,,,
0,Urban,Female,22,Bengali,Graduate,"> 60,000",With friends. colleagues or students,Graduate,Postgraduate,passive,...,100 plus 3pct,"30,000",save it,cheaper model,buy beverage,Multiple businesses/investment,No interest,less than 110,not sure,7.0


In [3]:
data = df.loc[:,["F1","F2","F3","F4","F5","F6","F7","K1","K2","K3","K4","K5","K6","K7","K8","K9","K10","K11","TOTAL"]]
data.head(1)

,F1,F2,F3,F4,F5,F6,F7,K1,K2,K3,K4,K5,K6,K7,K8,K9,K10,K11,TOTAL
id,,,,,,,,,,,,,,,,,,,
0,passive,passive,no record,Putting money in a savings/deposits account,no income,invest in assets,dont know,"Jewellery, Car, House, Art works",2 lakh,100 plus 3pct,"30,000",save it,cheaper model,buy beverage,Multiple businesses/investment,No interest,less than 110,not sure,7.0


In [4]:
from sklearn.preprocessing import LabelEncoder
for _ in range(18):
    data.iloc[:,_] = LabelEncoder().fit_transform(data.iloc[:,_])
for _ in range(len(data)):
    if data.iloc[_,-1] <= 6:
        data.iloc[_,-1] = 0
    else:
        data.iloc[_,-1] = 1
data.head()

,F1,F2,F3,F4,F5,F6,F7,K1,K2,K3,K4,K5,K6,K7,K8,K9,K10,K11,TOTAL
id,,,,,,,,,,,,,,,,,,,
0,1,1,0,10,4,0,4,16,0,0,2,0,0,0,0,0,1,2,1.0
1,0,1,2,3,4,1,3,28,3,2,3,0,0,0,1,1,3,2,0.0
2,1,1,1,29,0,1,3,2,3,2,3,0,1,1,0,1,3,2,0.0
3,0,1,2,10,4,1,5,26,1,0,1,0,0,0,0,0,3,0,1.0
4,1,0,2,10,0,1,1,16,1,0,3,0,0,1,0,0,2,3,1.0


In [5]:
X, y = data.iloc[:, :-1], data.iloc[:, -1]

In [59]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25)

In [60]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
states_dt = []
for _ in range(1,10):
    for __ in [10, 20, 40, 50, 80, 100]:
        dt = BaggingClassifier(DecisionTreeClassifier(max_depth= _, random_state=1), n_estimators=__, random_state=1)
        predict_dt = dt.fit(train_X, train_y).predict(val_X)
        states_dt.append((_, __, accuracy_score(predict_dt, val_y)))
max(states_dt, key = lambda x:x[2])

(5, 10, 0.9411764705882353)

- With maximum depth 5 and 10 bags we get 94.11% accuracy

In [61]:
from sklearn.naive_bayes import GaussianNB
states_gnb = []
for _ in range(1,9):
    gnb = GaussianNB(var_smoothing = 10**-_).fit(train_X, train_y)
    pred_gnb = gnb.predict(val_X)
    states_gnb.append((_, accuracy_score(pred_gnb, val_y)))
max(states_gnb, key = lambda x:x[1])

(2, 0.9019607843137255)

- Gaussian Naive Bayes is only giving 90.19% accuracy

In [62]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty="l2", random_state=1, max_iter = 1000)
pred_lr = lr.fit(train_X, train_y).predict(val_X)
accuracy_score(pred_lr, val_y)

0.8823529411764706

- Using logistic regression we're getting 88.23% accuracy

In [63]:
from sklearn.ensemble import RandomForestClassifier
states_rf = []
for _ in [10, 20, 40, 50, 80, 100]:
    rf = RandomForestClassifier(n_estimators=_, random_state=1)
    pred_rf = rf.fit(train_X, train_y).predict(val_X)
    states_rf.append((_, accuracy_score(pred_rf, val_y)))
max(states_rf, key = lambda x:x[1])

(20, 0.9215686274509803)

- Using random forest with 20 bags, we're gettng 92.15% accuracy

In [65]:
from sklearn.ensemble import VotingClassifier
dt = BaggingClassifier(DecisionTreeClassifier(max_depth= 4, random_state=1), n_estimators=10, random_state=1)
gnb = GaussianNB(var_smoothing = 10**-2)
lr = LogisticRegression(penalty="l2", random_state=1, max_iter = 1000)
rf = RandomForestClassifier(n_estimators=20, random_state=1)

final_model = VotingClassifier([("dt",dt), ("gnb",gnb), ("lr",lr), ("rf", rf)], voting = "hard")
final_pred = final_model.fit(train_X, train_y).predict(val_X)
accuracy_score(final_pred, val_y)

0.9607843137254902

- So with the above model we're getting around 96.07% accuracy on our test data. So the above model is best for predicting